<font color='blue'> **05__________________________________ Interactive_Variogram_Nugget_Effect** </font> 

In [ ]:
import os; import sys ; from pathlib import Path ; _R = Path("_R")   ; _R.mkdir(exist_ok=True); TA = Path('_R/05_Interactive_Variogram_Nugget_Effect_'); TA.mkdir(exist_ok=True) ; seed = 1
import numpy as np    ; import pandas as pd ; import matplotlib.pyplot as plt     ; import geostatspy.GSLIB as GSLIB
import math ; from ipywidgets import interactive ; from ipywidgets import widgets ; from ipywidgets import Layout ; from ipywidgets import Label ; from ipywidgets import VBox, HBox
REF1 = Path('E:/15_REPOS/02_Beta-Me/data/Krig/GeostatsPy')  
sys.path.append('E:/15_REPOS/02_Beta-Me/data/Krig/GeostatsPy')  ; from Variogram import GAM

In [2]:
NN = 200; mean = 0.20; stdev = 0.03; nlag = 100; pnoise = 0.5      ; Noise = NO = 100  ; Spatial_Continuity_Range = SP = 1
pnoise = widgets.FloatSlider(min=0.0,max = 1.0,value=NO,step = 0.05,description = 'Noise %',orientation='horizontal',style = {'description_width': 'initial'},layout=Layout(width='500px',height='30px'),continuous_update=False)
vrange = widgets.IntSlider(min=1,max=100,value=SP,step = 5,description = 'Spatial Continuity Range',orientation='horizontal',style = {'description_width': 'initial'},layout=Layout(width='500px',height='30px'),continuous_update=False)
ui = widgets.HBox([pnoise,vrange],)  ; l = widgets.Text(value='Prof. Michael Pyrcz, The University of Texas at Austin',layout=Layout(width='930px', height='30px')) ; ui2 = widgets.VBox([l,ui],)       
def run_plot(pnoise,vrange):               # ___________________________________________________________________________________________ DATA 
    global DA1 
    psignal = 1 - pnoise     ; np.random.seed(seed = seed)  ; DA0 = np.random.normal(loc=0.20,scale=0.03,size=NN+1000)                                              ; kern1 = np.ones(vrange)      
    data1 = np.convolve(DA0,kern1,mode='same')              ; DA1 = GSLIB.affine(data1[500:NN+500],mean,stdev)  ; DA1_rescale = GSLIB.affine(data1[500:NN+500],mean,stdev*math.sqrt(psignal))    
    DA1_noise = DA1_rescale + np.random.normal(loc=0.0,scale = stdev*math.sqrt(pnoise),size=NN)         ; DA1_noise = GSLIB.affine(DA1_noise,mean,stdev)
    fig = plt.figure(figsize=(4,3))        # ___________________________________________________________________________________________ DATA 
    spec = fig.add_gridspec(2, 3)          ; ax1 = fig.add_subplot(spec[0, :])  ; plt.plot(np.arange(1,NN+1),DA1,color='blue',alpha=0.3,lw=3,label='Original')
    plt.plot(np.arange(1,NN+1),DA1_noise,color='red',alpha=0.3,lw=3,label='Original + Noise')  ; plt.xlim([0,NN]); plt.ylim([mean-4*stdev,mean+4*stdev]) ; plt.ylabel('Porosity (%)'); plt.title('Data x Distance 1D')  ; plt.grid(); plt.legend(loc='upper right')
    ax2 = fig.add_subplot(spec[1, 0])      # ___________________________________________________________________________________________02 HISTO
    plt.hist(DA1,color='blue',alpha=0.3,edgecolor='black',bins=np.linspace(mean-4*stdev,mean+4*stdev,30),label='Original')     ; plt.hist(DA1_noise,color='red',alpha=0.3,edgecolor='black',bins=np.linspace(mean-4*stdev,mean+4*stdev,30),label='Original + Noise') 
    plt.xlim([mean-4*stdev,mean+4*stdev]); plt.ylim([0,30])  ; plt.xlabel('Porosity (%)'); plt.ylabel('Frequency'); plt.grid() ; plt.legend(loc='upper right')
    ax3 = fig.add_subplot(spec[1, 1])      # ___________________________________________________________________________________________03 Variance
    plt.pie([psignal, pnoise,],radius = 1, autopct='%1.1f%%',colors = ['#0000FF','#FF0000'], explode = [.02,.02],wedgeprops = {"edgecolor":"k",'linewidth':1,"alpha":0.3},) ; labels = ['Signal VAR','Noise',]; plt.legend(labels,loc='lower left')
    ax4 = fig.add_subplot(spec[1, 2])      # ___________________________________________________________________________________________04 _ VAR
    
    lag,gamma,npp = GAM(DA1,-9999,9999,1.0,1.0,0,1,nlag,1)  ; _,gamma_noise,_ = GAM(DA1_noise,-9999,9999,1.0,1.0,0,1,nlag,1)

    plt.scatter(lag,gamma,s=30,color='blue',alpha=0.3,edgecolor='black',label='Original')   ; plt.scatter(lag,gamma_noise,s=30,color='red',alpha=0.3,edgecolor='black',label='Original + Noise')
    plt.plot([0,nlag],[1.0,1.0],color='black',ls='--')                                      ; plt.xlim([0,nlag]); plt.ylim([0,2.0]); plt.grid(); plt.legend(loc='upper right')
    plt.xlabel('Lag Distance (h)'); plt.ylabel('Variogram')  ; plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=1.6, wspace=0.1, hspace=0.3); plt.show()

interactive_plot = widgets.interactive_output(run_plot, {'pnoise':pnoise,'vrange':vrange}) ; interactive_plot.clear_output(wait = True)  ; display(ui2, interactive_plot)

Output()